<a href="https://colab.research.google.com/github/zimazahra/Pemrograman_Phython/blob/main/Azima_Azzahra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Azima Azzahra
1101194376
TT-43-G4

In [1]:
#program utama
d = int(input("Masukkan jarak dalam kilometer adalh: "))
fc = int(input("Masukkan frekuensi 500 MHz-1500 MHz adalah: "))
area = input("Masukkan tipe area adalah: ")
ht = int(input("Masukkan tinggi antena pemancar 20-600 meter adalah: "))
hr = int(input("Masukkan tinggi antena penerima 1-50 meter adalah: "))

Masukkan jarak dalam kilometer adalh: 10
Masukkan frekuensi 500 MHz-1500 MHz adalah: 1000
Masukkan tipe area adalah: urban
Masukkan tinggi antena pemancar 20-600 meter adalah: 100
Masukkan tinggi antena penerima 1-50 meter adalah: 25


In [2]:
import numpy as np

In [3]:
NtL = lambda x: 10*np.log10(x)

In [4]:
LtN = lambda x: 10**(x/10)

In [5]:
def rumusc1 (z):
  if z in range (400,1500):
    return 69.55

  elif z in range (1500,2000):
    return 46.3

In [6]:
def rumusc2 (z):
  if z in range (400,1500):
    return 26.16

  elif z in range (1500,2000):
    return 33.9

In [7]:
def rumuscm (area,fc):
  if area ==  "urban":
    return 0
  if area == "suburban":
    return -2*(np.log10(fc/28)**2 - 5.4)
  if area == "open":
    return -4.78*(np.log10(fc))**2 + 18.33*np.log10(fc) - 40.94    

In [8]:
def rumusahr(z):
  return (1.1*np.log10(z)-0.7)*hr - (1.56*np.log10(z)-0.8)

In [9]:
c1 = rumusc1(fc)
c2 = rumusc2(fc)
cm = rumuscm(area,fc)
ahr = rumusahr(fc)

In [10]:
print(c1)
print(c2)
print(cm)
print(ahr)

69.55
26.16
0
61.12000000000001


In [11]:
pldb = c1 + c2*np.log10(fc) - 13.83*np.log10(ht) - ahr + (44.9-6.55*np.log10(ht))*np.log10(d) + cm
print("Pathloss adalah: " , pldb , " db")

Pathloss adalah:  91.04999999999998  db
